# Weather Mispricing Bot

A trading bot that:
1. Fetches live London weather from **4 sources** every 30s
2. Computes fair value for **WX_SPOT** and **WX_SUM**
3. Reads the orderbook from the CMI Exchange
4. Buys if `fair_value > best_ask + threshold`, sells if `fair_value < best_bid - threshold`
5. Adjusts confidence as the session progresses (more data = tighter threshold)

In [14]:
import math
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from threading import Thread, Lock
from bot_template import BaseBot, OrderBook, OrderRequest, Trade, Side, Order

---
## 1. Config

In [15]:
# ============================================================
# EXCHANGE
# ============================================================
TEST_URL = "http://ec2-52-49-69-152.eu-west-1.compute.amazonaws.com"
CHALLENGE_URL = "https://cmiexchange.replit.app"

EXCHANGE_URL = TEST_URL  # Switch to CHALLENGE_URL for competition
USERNAME = "weatherhamza"
PASSWORD = "weatherhamza"

# ============================================================
# API KEYS
# ============================================================
OWM_API_KEY  = "ad6b547d314dbfae707b72c101dbc740"
WAPI_API_KEY = "c5031f71c3874ac69e0191831262802"

# ============================================================
# TRADING PARAMS
# ============================================================
TRADE_VOLUME      = 5       # lots per trade
MAX_POSITION      = 20      # max net position (long or short)
THRESHOLD_WIDE    = 10      # early session: need big mispricing to act
THRESHOLD_TIGHT   = 5      # late session: act on smaller mispricings
WEATHER_POLL_SEC  = 30      # fetch weather every N seconds

LONDON_LAT, LONDON_LON = 51.5074, -0.1278

def c_to_f(c): 
    return c * 9/5 + 32

print(f"Exchange: {EXCHANGE_URL}")
print(f"Trading params: vol={TRADE_VOLUME}, max_pos={MAX_POSITION}, threshold={THRESHOLD_WIDE}→{THRESHOLD_TIGHT}")

Exchange: http://ec2-52-49-69-152.eu-west-1.compute.amazonaws.com
Trading params: vol=5, max_pos=20, threshold=10→5


---
## 2. Weather Fetchers (4 sources)

In [16]:
def fetch_open_meteo(past_steps=96, forecast_steps=96):
    """
    Source 1: Open-Meteo — THE settlement source. 15-min resolution.
    Returns DataFrame with time, temp_c, temp_f, humidity, t_x_h.
    """
    resp = requests.get("https://api.open-meteo.com/v1/forecast", params={
        "latitude": LONDON_LAT, "longitude": LONDON_LON,
        "minutely_15": "temperature_2m,relative_humidity_2m",
        "past_minutely_15": past_steps,
        "forecast_minutely_15": forecast_steps,
        "timezone": "Europe/London",
    }, timeout=10)
    resp.raise_for_status()
    m = resp.json()["minutely_15"]
    df = pd.DataFrame({
        "time": pd.to_datetime(m["time"]),
        "temp_c": m["temperature_2m"],
        "humidity": m["relative_humidity_2m"],
    })
    df["temp_f"] = c_to_f(df["temp_c"])
    df["temp_f_rounded"] = df["temp_f"].round()
    df["t_x_h"] = df["temp_f_rounded"] * df["humidity"]
    return df


def fetch_owm_current():
    """Source 2: OpenWeatherMap — current conditions."""
    resp = requests.get("https://api.openweathermap.org/data/2.5/weather", params={
        "lat": LONDON_LAT, "lon": LONDON_LON,
        "appid": OWM_API_KEY, "units": "metric",
    }, timeout=10)
    resp.raise_for_status()
    d = resp.json()
    temp_c = d["main"]["temp"]
    humidity = d["main"]["humidity"]
    temp_f = c_to_f(temp_c)
    return {"temp_f": temp_f, "temp_f_rounded": round(temp_f), "humidity": humidity,
            "t_x_h": round(temp_f) * humidity, "source": "owm"}


def fetch_owm_forecast():
    """Source 2b: OpenWeatherMap — 5-day/3-hour forecast."""
    resp = requests.get("https://api.openweathermap.org/data/2.5/forecast", params={
        "lat": LONDON_LAT, "lon": LONDON_LON,
        "appid": OWM_API_KEY, "units": "metric",
    }, timeout=10)
    resp.raise_for_status()
    rows = []
    for item in resp.json()["list"]:
        tc = item["main"]["temp"]
        h = item["main"]["humidity"]
        tf = c_to_f(tc)
        rows.append({"time": pd.Timestamp.fromtimestamp(item["dt"]),
                      "temp_f": tf, "temp_f_rounded": round(tf),
                      "humidity": h, "t_x_h": round(tf) * h})
    return pd.DataFrame(rows)


def fetch_wapi_current():
    """Source 3: WeatherAPI.com — current conditions."""
    resp = requests.get("https://api.weatherapi.com/v1/current.json", params={
        "key": WAPI_API_KEY, "q": f"{LONDON_LAT},{LONDON_LON}",
    }, timeout=10)
    resp.raise_for_status()
    d = resp.json()["current"]
    return {"temp_f": d["temp_f"], "temp_f_rounded": round(d["temp_f"]),
            "humidity": d["humidity"],
            "t_x_h": round(d["temp_f"]) * d["humidity"], "source": "wapi"}


def fetch_wapi_forecast(days=2):
    """Source 3b: WeatherAPI.com — hourly forecast."""
    resp = requests.get("https://api.weatherapi.com/v1/forecast.json", params={
        "key": WAPI_API_KEY, "q": f"{LONDON_LAT},{LONDON_LON}", "days": days,
    }, timeout=10)
    resp.raise_for_status()
    rows = []
    for day in resp.json()["forecast"]["forecastday"]:
        for hour in day["hour"]:
            tf = hour["temp_f"]
            h = hour["humidity"]
            rows.append({"time": pd.Timestamp(hour["time"]),
                          "temp_f": tf, "temp_f_rounded": round(tf),
                          "humidity": h, "t_x_h": round(tf) * h})
    return pd.DataFrame(rows)


def fetch_wttr_current():
    """Source 4: wttr.in — no API key needed."""
    resp = requests.get("https://wttr.in/London", params={"format": "j1"}, timeout=10)
    resp.raise_for_status()
    d = resp.json()["current_condition"][0]
    temp_f = float(d["temp_F"])
    humidity = int(d["humidity"])
    return {"temp_f": temp_f, "temp_f_rounded": round(temp_f), "humidity": humidity,
            "t_x_h": round(temp_f) * humidity, "source": "wttr"}


print("Fetchers defined. Testing...")
try:
    df_test = fetch_open_meteo(past_steps=4, forecast_steps=4)
    print(f"  Open-Meteo OK: {len(df_test)} rows")
except Exception as e:
    print(f"  Open-Meteo FAIL: {e}")

for name, fn in [("OWM", fetch_owm_current), ("WeatherAPI", fetch_wapi_current), ("wttr", fetch_wttr_current)]:
    try:
        r = fn()
        print(f"  {name} OK: T={r['temp_f_rounded']}°F, H={r['humidity']}%, T×H={r['t_x_h']}")
    except Exception as e:
        print(f"  {name} FAIL: {e}")

Fetchers defined. Testing...
  Open-Meteo OK: 8 rows
  OWM OK: T=44°F, H=73%, T×H=3212
  WeatherAPI OK: T=43°F, H=65%, T×H=2795
  wttr OK: T=45°F, H=57%, T×H=2565


---
## 3. Fair Value Engine

Computes fair values for WX_SPOT and WX_SUM by aggregating all sources.

**WX_SPOT** = `round(Temp_F) × Humidity` at Sunday 12:00  
**WX_SUM** = `Σ (round(Temp_F) × Humidity) / 100` over all 96 intervals (Sat 12:00 → Sun 12:00)

In [17]:
class FairValueEngine:
    """
    Aggregates weather data from multiple sources and computes 
    fair values for WX_SPOT and WX_SUM.
    
    Thread-safe: the bot's weather-polling thread writes,
    the trading logic reads.
    """
    
    def __init__(self, session_start: pd.Timestamp, session_end: pd.Timestamp):
        self.session_start = session_start
        self.session_end = session_end
        self.lock = Lock()
        
        # Latest state
        self.wx_spot_fv = None      # fair value for WX_SPOT
        self.wx_sum_fv = None       # fair value for WX_SUM
        self.confidence = 0.0       # 0 = start of session, 1 = end
        self.last_update = None
        self.source_readings = {}   # source_name -> latest reading
        
        # Open-Meteo full data (15-min resolution)
        self.om_data = None         # DataFrame from Open-Meteo
    
    def update(self):
        """
        Fetch all sources, compute fair values.
        Called every WEATHER_POLL_SEC seconds by the polling thread.
        """
        now = pd.Timestamp.now(tz="Europe/London")
        
        # --- Fetch all sources ---
        readings = {}  # source -> {temp_f_rounded, humidity, t_x_h}
        
        # Source 1: Open-Meteo (most important — settlement source)
        try:
            self.om_data = fetch_open_meteo(past_steps=96, forecast_steps=96)
            # Latest observation (most recent past timestamp)
            now_naive = now.tz_localize(None)
            past = self.om_data[self.om_data.time <= now_naive]
            if len(past) > 0:
                latest = past.iloc[-1]
                readings["open_meteo"] = {
                    "temp_f_rounded": latest["temp_f_rounded"],
                    "humidity": latest["humidity"],
                    "t_x_h": latest["t_x_h"],
                }
        except Exception as e:
            print(f"  [FV] Open-Meteo error: {e}")
        
        # Source 2: OWM
        try:
            readings["owm"] = fetch_owm_current()
        except Exception as e:
            print(f"  [FV] OWM error: {e}")
        
        # Source 3: WeatherAPI
        try:
            readings["wapi"] = fetch_wapi_current()
        except Exception as e:
            print(f"  [FV] WeatherAPI error: {e}")
        
        # Source 4: wttr.in
        try:
            readings["wttr"] = fetch_wttr_current()
        except Exception as e:
            print(f"  [FV] wttr error: {e}")
        
        if not readings:
            print("  [FV] WARNING: No sources available!")
            return
        
        # --- Compute WX_SPOT fair value ---
        # Use Open-Meteo FORECAST for settlement time (Sunday 12:00)
        spot_estimates = []
        spot_weights = []
        
        if self.om_data is not None:
            settlement_naive = self.session_end.tz_localize(None)
            diffs = (self.om_data.time - settlement_naive).abs()
            closest = self.om_data.iloc[diffs.argsort().iloc[0]]
            spot_estimates.append(closest["t_x_h"])
            spot_weights.append(0.50)  # highest weight: settlement source
        
        # Also use OWM forecast for settlement time
        try:
            df_owm_fc = fetch_owm_forecast()
            settlement_naive = self.session_end.tz_localize(None)
            diffs = (df_owm_fc.time - settlement_naive).abs()
            closest_owm = df_owm_fc.iloc[diffs.argsort().iloc[0]]
            spot_estimates.append(closest_owm["t_x_h"])
            spot_weights.append(0.25)
        except:
            pass
        
        # WeatherAPI forecast
        try:
            df_wapi_fc = fetch_wapi_forecast(days=2)
            settlement_naive = self.session_end.tz_localize(None)
            diffs = (df_wapi_fc.time - settlement_naive).abs()
            closest_wapi = df_wapi_fc.iloc[diffs.argsort().iloc[0]]
            spot_estimates.append(closest_wapi["t_x_h"])
            spot_weights.append(0.25)
        except:
            pass
        
        # --- Compute WX_SUM fair value ---
        sum_fv = None
        if self.om_data is not None:
            start_naive = self.session_start.tz_localize(None)
            end_naive = self.session_end.tz_localize(None)
            now_naive = now.tz_localize(None)

            session_data = self.om_data[
                (self.om_data.time >= start_naive) &
                (self.om_data.time <= end_naive)
            ]

            if len(session_data) > 0:
                observed = session_data[session_data.time <= now_naive]
                forecast = session_data[session_data.time > now_naive]
                observed_sum = (observed["t_x_h"] / 100).sum()
                forecast_sum = (forecast["t_x_h"] / 100).sum()
                sum_fv = observed_sum + forecast_sum
            elif spot_estimates:
                # Session beyond forecast window — use locally computed spot FV
                w = np.array(spot_weights)
                w = w / w.sum()
                local_spot_fv = np.average(spot_estimates, weights=w)
                sum_fv = 96 * local_spot_fv / 100
                        
                # --- Session progress / confidence ---
                total_duration = (self.session_end - self.session_start).total_seconds()
                elapsed = (now - self.session_start).total_seconds()
                self.confidence = max(0.0, min(1.0, elapsed / total_duration))
        
        # --- Store results (thread-safe) ---
        with self.lock:
            if spot_estimates:
                w = np.array(spot_weights)
                w = w / w.sum()
                self.wx_spot_fv = np.average(spot_estimates, weights=w)
            
            self.wx_sum_fv = sum_fv
            self.source_readings = readings
            self.last_update = now
    
    def get_state(self):
        """Thread-safe read of current fair values."""
        with self.lock:
            return {
                "wx_spot_fv": self.wx_spot_fv,
                "wx_sum_fv": self.wx_sum_fv,
                "confidence": self.confidence,
                "last_update": self.last_update,
                "n_sources": len(self.source_readings),
            }
    
    def get_threshold(self):
        """Dynamic threshold: wide at start, tight at end."""
        return THRESHOLD_WIDE + (THRESHOLD_TIGHT - THRESHOLD_WIDE) * self.confidence


print("FairValueEngine defined.")

FairValueEngine defined.


---
## 4. The Trading Bot

Extends `BaseBot` from the template. Architecture:

```
┌─────────────────────┐
│  Weather Thread      │  ← fetches 4 sources every 30s
│  → updates FV Engine │     computes fair_value_spot, fair_value_sum
└─────────┬───────────┘
          │ writes
          ▼
┌─────────────────────┐
│  FairValueEngine     │  ← thread-safe state
│  wx_spot_fv          │
│  wx_sum_fv           │
│  confidence          │
└─────────┬───────────┘
          │ reads
          ▼
┌─────────────────────┐
│  Trading Loop        │  ← runs every N seconds
│  1. read FV Engine   │     gets latest fair values
│  2. read orderbook   │     gets best bid/ask from exchange
│  3. compare & trade  │     BUY / SELL / HOLD
└─────────────────────┘
```

In [18]:
class WeatherBot(BaseBot):
    """
    Trades WX_SPOT and WX_SUM based on multi-source weather fair value.
    """
    
    def __init__(self, cmi_url, username, password, session_start, session_end):
        super().__init__(cmi_url, username, password)
        
        self.fv_engine = FairValueEngine(session_start, session_end)
        self._weather_thread = None
        self._running = False
        
        # Track latest orderbooks (updated by SSE)
        self._orderbooks = {}  # product -> OrderBook
        self._ob_lock = Lock()
    
    # ── SSE callbacks ──
    
    def on_orderbook(self, orderbook: OrderBook):
        """Called by SSE stream on every orderbook update."""
        with self._ob_lock:
            self._orderbooks[orderbook.product] = orderbook
    
    def on_trades(self, trade: Trade):
        """Called by SSE stream on every trade."""
        if trade.buyer == self.username or trade.seller == self.username:
            side = "BOUGHT" if trade.buyer == self.username else "SOLD"
            print(f"  💰 FILL: {side} {trade.volume}x {trade.product} @ {trade.price}")
    
    # ── Weather polling ──
    
    def _weather_loop(self):
        """Background thread: fetch weather, update fair values."""
        while self._running:
            try:
                self.fv_engine.update()
                state = self.fv_engine.get_state()
                spot_str = f"{state['wx_spot_fv']:.0f}" if state['wx_spot_fv'] is not None else "?"
                sum_str = f"{state['wx_sum_fv']:.0f}" if state['wx_sum_fv'] is not None else "?"
                print(f"  🌤  Weather updated: SPOT_FV={spot_str}  SUM_FV={sum_str}"
                    f"  conf={state['confidence']:.1%}  sources={state['n_sources']}")
            except Exception as e:
                print(f"  [Weather] Error: {e}")
            
            time.sleep(WEATHER_POLL_SEC)
    
    # ── Helpers ──
    
    def _get_best_bid_ask(self, product):
        """Read best bid/ask from cached orderbook (from SSE), excluding own orders."""
        
            
        with self._ob_lock:
            ob = self._orderbooks.get(product)
        
        # Always fall back to REST if SSE orderbook is stale/empty
        if ob is None or (not ob.buy_orders and not ob.sell_orders):
            try:
                ob = self.get_orderbook(product)
                with self._ob_lock:
                    self._orderbooks[product] = ob  # update cache
            except:
                return None, None
        
        # Best bid = highest buy order (excluding our own volume)
        best_bid = None
        for o in ob.buy_orders:
            market_vol = o.volume - o.own_volume
            if market_vol > 0:
                best_bid = o.price
                break
        
        # Best ask = lowest sell order (excluding our own volume)
        best_ask = None
        for o in ob.sell_orders:
            market_vol = o.volume - o.own_volume
            if market_vol > 0:
                best_ask = o.price
                break
        
        return best_bid, best_ask
    
    def _get_mid(self, product):
        """Get mid-price for a product."""
        bid, ask = self._get_best_bid_ask(product)
        if bid is not None and ask is not None:
            return (bid + ask) / 2
        return None
    
    # ── Core trading logic ──
    
    def _evaluate_and_trade(self, product, fair_value):
        """
        Compare fair value vs market for one product.
        If mispricing > threshold, send an order.
        """
        if fair_value is None:
            return
        
        best_bid, best_ask = self._get_best_bid_ask(product)
        if best_bid is None or best_ask is None:
            return
        
        threshold = self.fv_engine.get_threshold()
        positions = self.get_positions()
        current_pos = positions.get(product, 0)
        
        mid = (best_bid + best_ask) / 2
        
        # --- BUY signal: our FV is above the best ask ---
        if fair_value > best_ask + threshold:
            if current_pos < MAX_POSITION:
                vol = min(TRADE_VOLUME, MAX_POSITION - current_pos)
                if vol > 0:
                    print(f"  🟢 BUY {product}: FV={fair_value:.0f} > ask={best_ask}+{threshold:.0f}")
                    self.send_order(OrderRequest(product, best_ask, Side.BUY, vol))
                    
                    
            if current_pos >= MAX_POSITION and fair_value < best_bid - threshold/2:
                # Take profit on half position even if not a full sell signal
                vol = min(TRADE_VOLUME, current_pos // 2)
                self.send_order(OrderRequest(product, best_bid, Side.SELL, vol))
        
        # --- SELL signal: our FV is below the best bid ---
        elif fair_value < best_bid - threshold:
            if current_pos > -MAX_POSITION:
                vol = min(TRADE_VOLUME, MAX_POSITION + current_pos)
                if vol > 0:
                    print(f"  🔴 SELL {product}: FV={fair_value:.0f} < bid={best_bid}-{threshold:.0f}")
                    self.send_order(OrderRequest(product, best_bid, Side.SELL, vol))
        
        else:
            # No mispricing — could optionally place passive orders around FV
            pass
    
    # ── Main loop ──
    
    def run(self, interval=5):
        """
        Main entry point. Starts everything and loops.
        
        Args:
            interval: seconds between trading decisions
        """
        print("="*60)
        print("  WEATHER MISPRICING BOT")
        print("="*60)
        
        # 1. Start SSE stream (orderbook + trade updates)
        self.start()
        print("  SSE stream started.")
        
        # 2. Start weather polling thread
        self._running = True
        self._weather_thread = Thread(target=self._weather_loop, daemon=True)
        self._weather_thread.start()
        print(f"  Weather polling started (every {WEATHER_POLL_SEC}s).")
        
        # 3. Wait for first weather update
        print("  Waiting for first weather data...")
        while self.fv_engine.last_update is None:
            time.sleep(1)
        print("  Got first weather data. Trading loop starting.\n")
        
        # 4. Trading loop
        try:
            while True:
                state = self.fv_engine.get_state()
                
                # Print status
                spot_mid = self._get_mid("WX_SPOT")
                sum_mid = self._get_mid("WX_SUM")
                positions = self.get_positions()
                
                spot_pos = positions.get("WX_SPOT", 0)
                sum_pos = positions.get("WX_SUM", 0)
                threshold = self.fv_engine.get_threshold()
                
                sum_fv_str = f"{state['wx_sum_fv']:.0f}" if state['wx_sum_fv'] is not None else "?"

                print(f"  [{datetime.now().strftime('%H:%M:%S')}] "
                    f"SPOT: FV={state['wx_spot_fv']:.0f} mid={spot_mid} pos={spot_pos} | "
                    f"SUM: FV={sum_fv_str} mid={sum_mid} pos={sum_pos} | "
                    f"thr={threshold:.0f} conf={state['confidence']:.0%}")
                
                # Evaluate and trade
                self._evaluate_and_trade("WX_SPOT", state["wx_spot_fv"])
                self._evaluate_and_trade("WX_SUM", state["wx_sum_fv"])
                
                time.sleep(interval)
                
        except KeyboardInterrupt:
            print("\n  Stopping...")
        finally:
            self._running = False
            self.cancel_all_orders()
            self.stop()
            print("  Bot stopped. Final positions:", self.get_positions())
            print("  PnL:", self.get_pnl())


print("WeatherBot defined.")

WeatherBot defined.


---
## 5. Run the Bot

In [19]:
# ============================================================
# SESSION TIMES — change these to match your actual session!
# The exchange opens Saturday 14:30, closes Sunday 12:30
# But settlement window is Saturday 12:00 → Sunday 12:00
# ============================================================
SESSION_START = pd.Timestamp("2026-03-07 12:00", tz="Europe/London")
SESSION_END   = pd.Timestamp("2026-03-08 12:00", tz="Europe/London")

bot = WeatherBot(
    cmi_url=EXCHANGE_URL,
    username=USERNAME,
    password=PASSWORD,
    session_start=SESSION_START,
    session_end=SESSION_END,
)

# Run! (Ctrl+C to stop)
bot.run(interval=5)

  WEATHER MISPRICING BOT
  SSE stream started.
  Weather polling started (every 30s).
  Waiting for first weather data...
  🌤  Weather updated: SPOT_FV=3975  SUM_FV=3816  conf=0.0%  sources=4
  Got first weather data. Trading loop starting.

  [20:53:18] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:23] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:28] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:33] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:39] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:44] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  🌤  Weather updated: SPOT_FV=3975  SUM_FV=3816  conf=0.0%  sources=4
  [20:53:49] SPOT: FV=3975 mid=None pos=-20 | SUM: FV=3816 mid=3729.0 pos=20 | thr=10 conf=0%
  [20:53:55] SPOT: FV=3975 mid